In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def reshape_to_4(array):
    return array.reshape(-1, 4)

train_predR = reshape_to_4(pd.read_csv('X_ensembleRSpred.csv').values)
train_predF = reshape_to_4(pd.read_csv('X_ensembleFSpred.csv').values)
train_LSTMpredR = reshape_to_4(pd.read_csv('X_ensembleRMpred.csv').values)
train_LSTMpredF = reshape_to_4(pd.read_csv('X_ensembleFMpred.csv').values)

val_predR = reshape_to_4(pd.read_csv('vrealMpred.csv').values)
val_predF = reshape_to_4(pd.read_csv('vfakeSpred.csv').values)
val_LSTMpredR = reshape_to_4(pd.read_csv('vrealMpred.csv').values)
val_LSTMpredF = reshape_to_4(pd.read_csv('vfakeMpred.csv').values)

test_predR = reshape_to_4(pd.read_csv('trealSpred.csv').values)
test_predF = reshape_to_4(pd.read_csv('tfakeSpred.csv').values)
test_LSTMpredR = reshape_to_4(pd.read_csv('trealMpred.csv').values)
test_LSTMpredF = reshape_to_4(pd.read_csv('tfakeMpred.csv').values)


In [2]:
X_train = np.vstack((np.hstack((train_predR, train_LSTMpredR)),  
                          np.hstack((train_predF, train_LSTMpredF))))  

X_val = np.vstack((np.hstack((val_predR, val_LSTMpredR)),  
                         np.hstack((val_predF, val_LSTMpredF))))  

X_test = np.vstack((np.hstack((test_predR, test_LSTMpredR)),  
                         np.hstack((test_predF, test_LSTMpredF))))  


y_train = np.vstack((np.ones((train_predR.shape[0], 1)),  
                          np.zeros((train_predF.shape[0], 1))))  

y_val = np.vstack((np.ones((val_predR.shape[0], 1)),  
                          np.zeros((val_predF.shape[0], 1))))  

y_test = np.vstack((np.ones((test_predR.shape[0], 1)),  
                         np.zeros((test_predF.shape[0], 1))))  

In [3]:
# from tensorflow.keras.callbacks import Callback

# class ValidationCallback(Callback):
#     def __init__(self, val_data, val_labels):
#         self.val_data = val_data
#         self.val_labels = val_labels

#     def on_epoch_end(self, epoch, logs=None):
#         val_loss, val_acc = self.model.evaluate(self.val_data, self.val_labels, verbose=0)
#         print(f"\nEpoch {epoch+1}: Test Loss: {val_loss:.4f}, Test Acc: {val_acc:.4f}")

In [12]:
mlp_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(1, activation='sigmoid')
])
optimizer = Adam(learning_rate=0.0005)

mlp_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

validation_callback = ValidationCallback(X_test, y_test)

# mlp_model.fit(X_train, y_train, 
#               epochs=20, batch_size=512, 
#               validation_data=(X_val, y_val),
#               verbose=1,
#               callbacks=[validation_callback])

mlp_model.fit(X_train, y_train, 
              epochs=15, batch_size=512, 
              validation_data=(X_val, y_val),
              verbose=1)


mlp_accuracy = mlp_model.evaluate(X_test, y_test, verbose=0)[1]

print(f"Точність Покращеної MLP: {mlp_accuracy * 100:.2f}%")

Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8487 - loss: 0.4705 - val_accuracy: 0.5514 - val_loss: 0.6977
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9508 - loss: 0.2263 - val_accuracy: 0.6397 - val_loss: 0.6658
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9737 - loss: 0.1452 - val_accuracy: 0.7048 - val_loss: 0.6359
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9840 - loss: 0.1221 - val_accuracy: 0.7649 - val_loss: 0.6079
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9872 - loss: 0.1047 - val_accuracy: 0.8089 - val_loss: 0.5824
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9910 - loss: 0.0962 - val_accuracy: 0.8327 - val_loss: 0.5580
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9885 - loss: 0.0950 - val_accuracy: 0.8513 - val_loss: 0.5348
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9924 - loss: 0.0856 - val_accuracy: 0.8696 - val_loss: 0.5125
Epoch 9

In [13]:
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train.ravel())

svm_acc1 = svm_model.score(X_val, y_val)
svm_acc2 = svm_model.score(X_test, y_test)
print(f"Точність SVM на валідаційних: {svm_acc1 * 100:.2f}%")
print(f"Точність SVM на тестових: {svm_acc2 * 100:.2f}%")

Точність SVM на валідаційних: 94.45%
Точність SVM на тестових: 77.02%


In [15]:
rf_model = RandomForestClassifier(n_estimators=10, max_depth=8, random_state=42)
rf_model.fit(X_train, y_train.ravel())

rf_acc1 = rf_model.score(X_val, y_val)
rf_acc2 = rf_model.score(X_test, y_test)
print(f"Точність Random Forest на валідаційних: {rf_acc1 * 100:.2f}%")
print(f"Точність Random Forest на тестових: {rf_acc2 * 100:.2f}%")

Точність Random Forest на валідаційних: 94.39%
Точність Random Forest на тестових: 73.90%
